In [29]:
from django.test import TestCase
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine



Server ='bknew-sql02'
Database = 'Bradken_Data_Warehouse'
Driver = 'ODBC Driver 17 for SQL Server'
Database_Con = f'mssql://@{Server}/{Database}?driver={Driver}'
engine = create_engine(Database_Con)
con_Bradken_Data_Warehouse_Epicor = engine.connect()

df_Epicor_PowerBI_Products =pd.read_sql_query("Select * from PowerBI.Products where RowEndDate IS NULL",con_Bradken_Data_Warehouse_Epicor)
df_Epicor_PowerBI_Site =pd.read_sql_query("Select * from PowerBI.Site where RowEndDate IS NULL",con_Bradken_Data_Warehouse_Epicor)
df_Epicor_PowerBI_Sales_Orders =pd.read_sql_query("Select * from PowerBI.SalesOrders",con_Bradken_Data_Warehouse_Epicor)

df = df_Epicor_PowerBI_Sales_Orders.merge(df_Epicor_PowerBI_Products,how='left', left_on='skProductId', right_on='skProductId')



df = df.merge(df_Epicor_PowerBI_Site, how='left', left_on='skSiteId', right_on='skSiteId')

df = df.filter(items=['ProductKey','OnOrderQty', 'SiteName'])

df = df.query("OnOrderQty>0")
df = df.drop_duplicates(['ProductKey'])
df



,ProductKey,OnOrderQty,SiteName
75,FS1002,460.0,FOR1
78,BP9G100X312UNC,24.0,POB1
420,GB3003,500.0,MfgSys
433,B129C30X220M,5.0,POB1
437,AL04,1.0,MTT1
...,...,...,...
45466,C204-0323N,1.0,BAS1
45467,C204-0323L,2.0,BAS1
45470,M48CW,824.0,DTC1
45471,BK58435-2-007A,44.0,DTC1


In [188]:
from django.test import TestCase
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine



Server ='bknew-sql02'
Database = 'Bradken_Data_Warehouse'
Driver = 'ODBC Driver 17 for SQL Server'
Database_Con = f'mssql://@{Server}/{Database}?driver={Driver}'
engine = create_engine(Database_Con)
con_Bradken_Data_Warehouse_Epicor = engine.connect()

df_Epicor_PowerBI_Products =pd.read_sql_query("Select * from PowerBI.Products where RowEndDate IS NULL",con_Bradken_Data_Warehouse_Epicor)
df_Epicor_PowerBI_Site =pd.read_sql_query("Select * from PowerBI.Site where RowEndDate IS NULL",con_Bradken_Data_Warehouse_Epicor)
df_Epicor_PowerBI_Heat_Products =pd.read_sql_query("Select * from PowerBI.HeatProducts",con_Bradken_Data_Warehouse_Epicor)
df_Epicor_PowerBI_Dates =pd.read_sql_query("Select * from PowerBI.Dates",con_Bradken_Data_Warehouse_Epicor)
df = df_Epicor_PowerBI_Heat_Products.merge(df_Epicor_PowerBI_Products, how='left',left_on='skProductId', right_on='skProductId')
df = df.merge(df_Epicor_PowerBI_Site, how='left',left_on='SkSiteId', right_on='skSiteId')
df = df.filter(items=['TapTime','TappedMass', 'ProductGroup', 'SiteName','DressMass','CastQty'])
df = df.merge(df_Epicor_PowerBI_Dates, how='left', left_on='TapTime', right_on='DateValue')
df = df.filter(items=['TapTime','TappedMass', 'ProductGroup', 'SiteName','FiscalMonthOfYearDescJapan','DressMass','CastQty','YearMonth'])
df = df.query('SiteName == "WOD1" ')
df['Tonnes']=df['CastQty'].mul(df['DressMass'])/1000

df = df[['YearMonth','Tonnes']].groupby('YearMonth').sum().sort_values('YearMonth')
df = df['Tonnes'].astype(int)

df

YearMonth
202407.0      2
202408.0    284
202409.0    247
202410.0    284
202411.0    226
202412.0    211
202501.0    240
202502.0     74
Name: Tonnes, dtype: int64

In [182]:
df.index

Index([202407.0, 202408.0, 202409.0, 202410.0, 202411.0, 202412.0, 202501.0,
       202502.0],
      dtype='float64', name='YearMonth')

In [178]:
import os  
from pathlib import Path  









In [186]:
df

,TapTime,TappedMass,ProductGroup,SiteName,FiscalMonthOfYearDescJapan,DressMass,CastQty,YearMonth,Tonnes
0,2024-10-07,3600.0,MINGET,WOD1,Oct FY2024,11.2,26.0,202410.0,0.2912
1,2024-10-07,3600.0,MINGET,WOD1,Oct FY2024,11.2,12.0,202410.0,0.1344
2,2024-10-07,3600.0,MINGET,WOD1,Oct FY2024,47.0,2.0,202410.0,0.0940
3,2024-10-07,3600.0,MINGET,WOD1,Oct FY2024,47.0,2.0,202410.0,0.0940
4,2024-10-07,3600.0,MINGET,WOD1,Oct FY2024,47.0,2.0,202410.0,0.0940
...,...,...,...,...,...,...,...,...,...
5909,2024-10-29,3650.0,MINGET,WOD1,Oct FY2024,76.0,4.0,202410.0,0.3040
5910,2024-10-29,3650.0,MINGET,WOD1,Oct FY2024,76.0,5.0,202410.0,0.3800
5911,2024-10-29,3650.0,MINGET,WOD1,Oct FY2024,76.0,2.0,202410.0,0.1520
5912,2024-10-29,3650.0,MINGET,WOD1,Oct FY2024,92.0,2.0,202410.0,0.1840
